In [ ]:
# For data
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import datetime
# Use to grab data from the web(HTTP capabilities)
import requests
from io import StringIO
import io
import re

# For timestamps
from datetime import datetime

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from __future__ import division

# This is the url link for the poll data in csv form
url = "http://elections.huffingtonpost.com/pollster/"
file_2012="2012-general-election-romney-vs-obama.csv"
file_2016="2016-general-election-trump-vs-clinton.csv"
# Use requests to get the information in text form
source_1 = requests.get(url+file_2012).text
source_2 = requests.get(url+file_2016).text
# Use StringIO to avoid an IO error with pandas
poll_data_1 = StringIO(source_1) 
poll_data_2 = StringIO(source_2) 

# Set poll data as pandas DataFrame
poll_df_1 = pd.read_csv(poll_data_1)
# Set poll data as pandas DataFrame
poll_df_2 = pd.read_csv(poll_data_2)

# Let's get a glimpse at the data
poll_df_1.info()

poll_df_1['Population'].unique()
poll_df_2['Population'].unique()

df1=poll_df_1[['Population','Obama','Romney','Undecided','Other']]
df1gb=df1.groupby('Population')
df1gb.count()

df1[df1['Population']=='Likely Voters - Republican']

df1=df1.replace(to_replace='Likely Voters - Republican', value='Likely Voters', inplace=False, axis=None)
df1gb=df1.groupby('Population')
df1gb.count()

poll_df_2['Population'].unique()
df2=poll_df_2[['Population','Trump','Clinton','Undecided','Other']]
df2gb=df2.groupby('Population')
df2gb.count()

poll_df_1=poll_df_1.replace(to_replace='Likely Voters - Republican', value='Likely Voters', inplace=False, axis=None)

poll_df_2LV=poll_df_2[poll_df_2['Population']=='Likely Voters']
poll_df_2RV=poll_df_2[poll_df_2['Population']=='Registered Voters']
poll_df_2AD=poll_df_2[poll_df_2['Population']=='Adults']
full_df_2=poll_df_2LV.append(poll_df_2RV)
full_df_2=full_df_2.append(poll_df_2AD)

df=full_df_2[['Population','Trump','Clinton','Undecided','Other']]
df=df.groupby('Population')
df.count()

poll_df_1['Population'].unique()
poll_df_2['Population'].unique()
full_df_2['Population'].unique()

#checking for skewed 2012 data
Romney_big=poll_df_1[poll_df_1['Romney']>60]
Romney_big[['Pollster','Population','Mode','Romney']]

Obama_big=poll_df_1[poll_df_1['Obama']>60]
Obama_big[['Pollster','Population','Mode','Obama']]

skewed_data=Obama_big.append(Romney_big)
skewed_data

poll_df_1.rename(columns={'Obama': 'Candidate_1','Romney':'Candidate_2'}, inplace=True)

#Investigation of suspicious standard errors from 2016 data

trump_big=full_df_2[full_df_2['Trump']>60]
trump_big[['Pollster','Population','Mode','Trump']]

hillary_big=full_df_2[full_df_2['Clinton']>60]
hillary_big[['Pollster','Population','Mode','Clinton']]

skewed_data=hillary_big.append(trump_big)
#skewed_data['Trump'].mean(),skewed_data['Clinton'].mean()
#average Trump=43.205329153605014, Clinton=45.56583072100314
skewed_data

#poll_df_2=poll_df_2[poll_df_2['Clinton']<61]
#poll_df_2=poll_df_2[poll_df_2['Trump']<61]
#poll_df_2.info()

# Used for producing simple graph
#df1=DataFrame(skewed_data,columns=['Trump'])
#df1['One']=1
#df1=df1.groupby('Trump').count()
#df1
#sns_plot=sns.factorplot('One',data=df1, kind='count', size=5, aspect=2)
#df2=DataFrame(skewed_data,columns=['Clinton'])
#df2['One']=1
#df2=df2.groupby('Clinton').count()
#df2

full_df_2.rename(columns={'Trump': 'Candidate_1','Clinton':'Candidate_2'}, inplace=True)
full_data=poll_df_1.append(full_df_2)

full_data.info()
full_data.sort_values('End Date').head()

full_data['Number of Observations'].sum()
full_data['Number of Observations'].mean()

# Factorplot the affiliation 2012 and 2016
sns_plot=sns.factorplot('Affiliation',data=full_data,kind='count')
sns_plot.savefig("affil_2012_2016.jpg")

# Factorplot the affiliation by Population 2012 and 2016
sns_plot=sns.factorplot('Affiliation',data=full_data,hue='Population',kind='count')
sns_plot.savefig("affil_2012_2016_breakdown.jpg")

# Factorplot the affiliation by Population 2012
sns_plot=sns.factorplot('Affiliation',data=full_data[full_data['End Date']<'2014-01-01'],kind='count')
sns_plot.savefig("affil_2012.jpg")

# Factorplot the affiliation by Population 2012
#sns_plot=sns.factorplot('Affiliation',data=full_data[full_data['End Date']<'2013-01-01'],hue='Population',kind='count')
#sns_plot.savefig("affil_2012_breakdown.jpg")

# Factorplot the affiliation by Population 2016
sns_plot=sns.factorplot('Affiliation',data=full_data[full_data['End Date']>'2014-01-01'],kind='count')
sns_plot.savefig("affil_2016.jpg")

# Factorplot the affiliation by Population 2016
#sns_plot=sns.factorplot('Affiliation',data=full_data[full_data['End Date']>'2013-01-01'],hue='Population',kind='count')
#sns_plot.savefig("affil_2016_breakdown.jpg")

# First we'll get the average
avg_2012 = pd.DataFrame(full_data[full_data['End Date']<'2013-01-01'].mean()) 
avg_2012.drop('Number of Observations',axis=0,inplace=True)
avg_2012.drop('Question Iteration',axis=0,inplace=True)


#avg_2012.drop('Other',axis=0,inplace=True)
#avg_2012.rename(index={'Candidate_1': 'Obama','Candidate_2': 'Romney'})

# After that let's get the error
std_2012 = pd.DataFrame(full_data[full_data['End Date']<'2013-01-01'].std())
std_2012.drop('Number of Observations',axis=0,inplace=True)
std_2012.drop('Question Iteration',axis=0,inplace=True)


#std_2012.drop('Other',axis=0,inplace=True)
std_2012.rename(index={'Candidate_1':'Obama','Candidate_2': 'Romney'},inplace=True)
#std_2012

std_2012


# First we'll get the average
#avg = pd.DataFrame(poll_df_2.mean()) 
#avg.drop('Number of Observations',axis=0,inplace=True)

# After that let's get the error
#std = pd.DataFrame(poll_df_2.std())
#std.drop('Number of Observations',axis=0,inplace=True)

# now plot using pandas built-in plot, with kind='bar' and yerr='std'
#avg.plot(yerr=std,kind='bar',legend=False)

#fig=avg.plot(yerr=std,kind='bar',legend=False)
#fig.plot()
#plt.savefig("Trump_Hillary.png")

avg_2012

# now plot using pandas built-in plot, with kind='bar' and yerr='std'
avg_2012.rename(index={'Candidate_1': 'Obama','Candidate_2': 'Romney'},inplace=True)

fig=avg_2012.plot(yerr=std_2012,kind='bar',legend=False)
fig.plot()
plt.savefig("Obama_Romney.png")

# Concatenate our Average and Std DataFrames
poll_avg_2012 = pd.concat([avg_2012,std_2012],axis=1)

#Rename columns
poll_avg_2012.columns = ['Average','STD']
poll_avg_2012.rename(index={'Candidate_1':'Obama','Candidate_2': 'Romney'},inplace=True)
#Show
poll_avg_2012

# First we'll get the average
avg_2016 = pd.DataFrame(full_data[full_data['End Date']>'2014-01-01'].mean()) 
avg_2016.drop('Number of Observations',axis=0,inplace=True)
avg_2016.drop('Question Iteration',axis=0,inplace=True)
#avg_2016.drop('Other',axis=0,inplace=True)
avg_2016.rename(index={'Candidate_1': 'Trump','Candidate_2': 'Clinton'},inplace=True)

# After that let's get the error
std_2016 = pd.DataFrame(full_data[full_data['End Date']>'2014-01-01'].std())
std_2016.drop('Number of Observations',axis=0,inplace=True)
std_2016.drop('Question Iteration',axis=0,inplace=True)
#std_2016.drop('Other',axis=0,inplace=True)
std_2016.rename(index={'Candidate_1': 'Trump','Candidate_2': 'Clinton'},inplace=True)

# now plot using pandas built-in plot, with kind='bar' and yerr='std'
avg_2016.rename(index={'Candidate_1': 'Trump','Candidate_2': 'Clinton'},inplace=True)
fig=avg_2016.plot(yerr=std_2016,kind='bar',legend=False)
fig.plot()
plt.savefig("Trump_Hillary.png")

# Concatenate our Average and Std DataFrames
poll_avg_2016 = pd.concat([avg_2016,std_2016],axis=1)

#Rename columns
poll_avg_2016.columns = ['Average','STD']
poll_avg_2016.rename(index={'Candidate_1': 'Trump','Candidate_2': 'Clinton'},inplace=True)
#Show
poll_avg_2016

# Quick plot of sentiment in the polls versus time.
fig=full_data[full_data['End Date']<'2014-01-01'].plot(figsize=(12,4),x='End Date',y=['Candidate_1','Candidate_2','Undecided'],marker='o',linestyle='')
fig.plot()
plt.savefig("Obama_Romney_Undecided.png")

# Quick plot of sentiment in the polls versus time.
fig=full_data[full_data['End Date']>'2014-01-01'].plot(figsize=(12,4),x='End Date',y=['Candidate_1','Candidate_2','Undecided'],marker='o',linestyle='')
fig.plot()
plt.savefig("Hillary_Trump_Undecided.png")

# Create a new column for the difference between the two candidates
full_data['Difference'] = (full_data.Candidate_1 - full_data.Candidate_2)/100
# Preview the new column
full_data[:10]

#Candidate 1 = Obama, Trump (2012, 2016) respectively

#Candidate 2 = Romney, Clinton (2012, 2016) respectively

# Set as_index=Flase to keep the 0,1,2,... index. Then we'll take the mean of the polls on that day.
full_data_groupby = full_data.groupby(['Start Date'],as_index=False).mean()

# Let's go ahead and see what this looks like
full_data_groupby.head()

# Plotting the difference in polls between Obama and Romney
fig = full_data_groupby[full_data_groupby['Start Date']<'2013-01-01'].plot('Start Date','Difference',figsize=(12,5),marker='o',linestyle='-',color='blue')
fig.plot()
plt.savefig("Romney_Obama_Overtime.png")

fig_sort=full_data_groupby[full_data_groupby['Start Date']<'2013-01-01'].sort_values('Difference')
DataFrame(fig_sort,columns=['Start Date','Candidate_1','Candidate_2','Number of Observations','Undecided','Difference','Pollster'])

# Plotting the difference in polls between Trump and Clinton
fig = full_data_groupby[full_data_groupby['Start Date']>'2013-01-01'].plot('Start Date','Difference',figsize=(12,5),marker='o',linestyle='-',color='red')
fig.plot()
plt.savefig("Clinton_Trump_Overtime.png")

# Set row count and xlimit list
row_in = 0
xlimit = []

# Cycle through dates until 2012-10 is found, then print row index
for date in full_data_groupby[full_data_groupby['Start Date']<'2013-01-01']['Start Date']:
    if date[0:7] == '2012-10':
        xlimit.append(row_in)
        row_in +=1
    else:
        row_in += 1
        
print(min(xlimit))
print(max(xlimit))

### Start with original figure
fig = full_data_groupby[full_data_groupby['Start Date']<'2013-01-01'].plot('Start Date','Difference',figsize=(12,4),marker='o',linestyle='-',color='blue',xlim=(90,120))

fig.plot()
plt.savefig("Romney_Obama_timeline_peak2.png")

### Start with original figure
fig = full_data_groupby[full_data_groupby['Start Date']<'2013-01-01'].plot('Start Date','Difference',figsize=(12,4),marker='o',linestyle='-',color='blue',xlim=(325,352))

# Now add the debate markers
plt.axvline(x=325+2, linewidth=2, color='grey')
plt.axvline(x=325+13, linewidth=2, color='grey')
plt.axvline(x=325+18, linewidth=2, color='grey')
#Oct 3rd, Oct 16, and Oct 22nd
fig.plot()
plt.savefig("Romney_Obama_timeline.png")

# Set row count and xlimit list
row_in = 0
xlimit = []

# Cycle through dates until 2012-10 is found, then print row index
for date in full_data_groupby[full_data_groupby['Start Date']>'2013-01-01']['Start Date']:
    if date[0:7] == '2016-09':
        xlimit.append(row_in)
        row_in +=1
    else:
        row_in += 1
        
print(min(xlimit))
print(max(xlimit))

# Start with original figure
fig = full_data_groupby[full_data_groupby['Start Date']>'2013-01-01'].plot('Start Date','Difference',figsize=(12,4),marker='o',linestyle='-',color='red',xlim=(224,262))

# Now add the debate markers
plt.axvline(x=224+4, linewidth=2, color='grey')
plt.axvline(x=209+31, linewidth=2, color='grey')
plt.axvline(x=209+40, linewidth=2, color='grey')
#sept. 26, oct. 4 (vice president), oct.9, oct. 19
#seperated by 5 days
#minimum on dates to find out what happened
fig.plot()
plt.savefig("Clinton_Trump_timeline.png")

######################################################
### WEB SCRAPER TO COMPILE NEWS ARTICLES FOR DATES ###
######################################################

from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series,DataFrame
import re

url = 'https://www.realclearpolitics.com/pm_update/2011/36/'
# Request content from web page
result = requests.get(url)
page = result.content

# Set as Beautiful Soup Object
soup=BeautifulSoup(page,"html.parser")

print(soup.title.text)
date=soup.findAll('div', {'style':"color:#666; font-size:15px; font-weight:bold; margin:0px; padding:10px 0px 4px 0px;"})

for d in date:
    print(d.get_text())
    
L_links=[]
for link in soup.findAll('a'):
    L_links.append(link.attrs)
    L_links.append(link.contents)
    
L_links=L_links[140:287]

links=[]
titles=[]
for count, i in enumerate(L_links):
    if count % 2 == 0:
        links.append(i)
    else:
        titles.append(i[0])  

justLinks=[]
count=0
for l in links:
    try:
        print(links[count]['href'])
        justLinks.append(links[count]['href'])
        count+=1
    except:
        print('Not a link')
        
pd.DataFrame(list(zip(titles, justLinks)),
              columns=['Titles','Links'])

for link in soup.find_all('a'):
    print(link.get('href')) 
